# Imports

In [1]:
import numpy as np
import _pickle as pickle
import gym
import time
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.initializers as initializers

# Model

In [3]:
initializer = initializers.GlorotNormal

x_in = layers.Input(shape = (6400,))
x = layers.Dense(200, kernel_initializer= initializer, activation="relu")(x_in)
x_out = layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(x_in, x_out)

model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["acc"])

In [4]:
env = gym.make("Pong-v0")

In [5]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [6]:
env.unwrapped.get_action_meanings()

# NOOP is the same as FIRE (standing still)
# LEFT is the same as LEFTFIRE (down)
# RIGHT is the same as RIGHTFIRE (up)

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 6400)]            0         
_________________________________________________________________
dense (Dense)                (None, 200)               1280200   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 1,280,401
Trainable params: 1,280,401
Non-trainable params: 0
_________________________________________________________________


In [8]:
env.unwrapped.get_keys_to_action()

{(): 0, (32,): 1, (100,): 2, (97,): 3, (32, 100): 4, (32, 97): 5}

# Preprocessing

In [9]:
def prepro(input_frame):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    input_frame = input_frame[34:194] # crop
    input_frame = input_frame[::2,::2,0] # downsample by factor of 2 (halves the resolution of the image)
    #This takes every other pixel in the image
    input_frame[input_frame == 144] = 0 # erase background (background type 1)
    input_frame[input_frame == 109] = 0 # erase background (background type 2)
    input_frame[input_frame != 0] = 1 # everything else (paddles, ball) just set to 1
    return input_frame.astype(np.float).ravel()

# Discounted Rewards

In [10]:
def discount_rewards(rewards):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(rewards)
    running_add = 0
    for t in reversed(range(0, rewards.size)):
        if rewards[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + rewards[t]
        discounted_r[t] = running_add
    return discounted_r

# Hyperparamters

In [13]:
render = True
prev_frame = None
game_dimensions = 80*80
gamma = 0.99
resume = False
batch_size = 10

# Main Code

In [26]:
observation = env.reset()
reward_sum = 0
epsilon = 0  # Epsilon greedy parameter
epsilon_min = 0.000001  # Minimum epsilon greedy parameter
epsilon_max = 1.0  # Maximum epsilon greedy parameter
epsilon_interval = (
    epsilon_max - epsilon_min
)
episode_number = 0
running_reward = None
ep_observations, ep_rewards, ep_gradient_log_ps, ep_actions = [], [], [], []
# Number of frames to take random action and observe output
epsilon_random_frames = 50000
# Number of frames for exploration
epsilon_greedy_frames = 1000000.0

if resume:
    model.load_weights("ModelWeights")

while True:
    if render:
        env.render()
    
    curr_frame = prepro(observation)
    change_in_frame = curr_frame - prev_frame if prev_frame is not None else np.zeros(game_dimensions)
    prev_frame = curr_frame
    ep_observations.append(change_in_frame)
    
    if np.random.random() < epsilon:
        action = np.random.randint(2, 4)
    else:
        change_in_frame = change_in_frame.reshape((1,6400))
        up_prob = model.predict(change_in_frame)
        if up_prob >= .5:
            action = 2
        else:
            action = 3
    
    observation, reward, done, _ = env.step(action) 
    
    y = 1 if action == 2 else 0
    
    try:
        ep_gradient_log_ps.append(y - up_prob)
    except:
        ep_gradient_log_ps.append(y)
    
    ep_actions.append(y)
    
    ep_rewards.append(reward)
    reward_sum += reward
    
    # Decay probability of taking random action
    epsilon -= epsilon_interval / epsilon_greedy_frames
    epsilon = max(epsilon, epsilon_min)
    
    if done:
        comb_ep_observations = np.vstack(ep_observations)
        comb_ep_gradient_log_ps = np.vstack(ep_gradient_log_ps)
        comb_ep_rewards = np.vstack(ep_rewards)
        comb_ep_actions = np.vstack(ep_actions)
        ep_observations, ep_gradient_log_ps, ep_rewards, ep_actions = [], [], [], []
        
        discounted_comb_ep_rewards = discount_rewards(comb_ep_rewards)
        discounted_comb_ep_rewards -= np.mean(discounted_comb_ep_rewards) 
        discounted_comb_ep_rewards /= np.std(discounted_comb_ep_rewards)
        
        test_data = []
        
        for action in range(len(comb_ep_actions)):
            if discounted_comb_ep_rewards[action] < 0:
                if comb_ep_actions[action] == 1:
                    test_data.append(0)
                else:
                    test_data.append(1)
            else:
                app = int(comb_ep_actions[action])
                test_data.append(app)

        comb_ep_gradient_log_ps = comb_ep_gradient_log_ps * discounted_comb_ep_rewards
        
        model.fit(comb_ep_observations, np.asarray(test_data), epochs = 50)
        
        print(np.unique(test_data))
        
        running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
        print ('resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward))
        reward_sum = 0
        observation = env.reset() # reset env
        prev_frame = None
        episode_number += 1
        test_data = []
        print("epsilon is: ", epsilon)
        
        if episode_number % batch_size == 0:
            
            # Back Propagation Code Goes Here
            
            model.save_weights("ModelWeights")
    
    if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
        print('ep %d: game finished, reward: %f' % (episode_number, reward) + ('' if reward == -1 else ' !!!!!!!!'))

ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
Epoch 1/50
32/32 [==============================] - 1s 3ms/step - loss: 100605.5196 - acc: 0.5131
Epoch 2/50
32/32 [==============================] - 0s 3ms/step - loss: 103989.2159 - acc: 0.4837
Epoch 3/50
32/32 [======

34/34 [==============================] - 0s 4ms/step - loss: 77528.4219 - acc: 0.4733
Epoch 18/50
34/34 [==============================] - 0s 4ms/step - loss: 77156.8828 - acc: 0.4697
Epoch 19/50
34/34 [==============================] - 0s 4ms/step - loss: 76785.3438 - acc: 0.4706
Epoch 20/50
34/34 [==============================] - 0s 4ms/step - loss: 76420.5078 - acc: 0.4724
Epoch 21/50
34/34 [==============================] - 0s 4ms/step - loss: 76055.3359 - acc: 0.4697
Epoch 22/50
34/34 [==============================] - 0s 4ms/step - loss: 75690.3984 - acc: 0.4715
Epoch 23/50
34/34 [==============================] - 0s 4ms/step - loss: 75323.3125 - acc: 0.4678
Epoch 24/50
34/34 [==============================] - 0s 4ms/step - loss: 74965.9688 - acc: 0.4678
Epoch 25/50
34/34 [==============================] - 0s 4ms/step - loss: 74610.9922 - acc: 0.4688
Epoch 26/50
34/34 [==============================] - 0s 4ms/step - loss: 74252.0234 - acc: 0.4669
Epoch 27/50
34/34 [=============

35/35 [==============================] - 0s 4ms/step - loss: 126387.0469 - acc: 0.4782
Epoch 42/50
35/35 [==============================] - 0s 4ms/step - loss: 125982.8125 - acc: 0.4764
Epoch 43/50
35/35 [==============================] - 0s 4ms/step - loss: 125581.0234 - acc: 0.4755
Epoch 44/50
35/35 [==============================] - 0s 4ms/step - loss: 125191.8438 - acc: 0.4737
Epoch 45/50
35/35 [==============================] - 0s 4ms/step - loss: 124792.5625 - acc: 0.4764
Epoch 46/50
35/35 [==============================] - 0s 4ms/step - loss: 124406.3125 - acc: 0.4737
Epoch 47/50
35/35 [==============================] - 0s 4ms/step - loss: 124016.2734 - acc: 0.4737
Epoch 48/50
35/35 [==============================] - 0s 4ms/step - loss: 123627.7734 - acc: 0.4719
Epoch 49/50
35/35 [==============================] - 0s 4ms/step - loss: 123234.9219 - acc: 0.4728
Epoch 50/50
35/35 [==============================] - 0s 4ms/step - loss: 122845.3438 - acc: 0.4701
[0 1]
resetting env. e

35/35 [==============================] - 0s 4ms/step - loss: 115578.0469 - acc: 0.4771
Epoch 6/50
35/35 [==============================] - 0s 3ms/step - loss: 114991.5547 - acc: 0.4771
Epoch 7/50
35/35 [==============================] - 0s 3ms/step - loss: 114418.4453 - acc: 0.4789
Epoch 8/50
35/35 [==============================] - 0s 3ms/step - loss: 113851.1406 - acc: 0.4798
Epoch 9/50
35/35 [==============================] - 0s 3ms/step - loss: 113307.9141 - acc: 0.4816
Epoch 10/50
35/35 [==============================] - 0s 3ms/step - loss: 112776.2188 - acc: 0.4843
Epoch 11/50
35/35 [==============================] - 0s 3ms/step - loss: 112257.8125 - acc: 0.4906
Epoch 12/50
35/35 [==============================] - 0s 3ms/step - loss: 111750.9688 - acc: 0.4978
Epoch 13/50
35/35 [==============================] - 0s 3ms/step - loss: 111248.0312 - acc: 0.4969
Epoch 14/50
35/35 [==============================] - 0s 3ms/step - loss: 110754.6719 - acc: 0.4978
Epoch 15/50
35/35 [=======

Epoch 29/50
35/35 [==============================] - 0s 4ms/step - loss: 93375.5078 - acc: 0.5295
Epoch 30/50
35/35 [==============================] - 0s 4ms/step - loss: 93005.1172 - acc: 0.5259
Epoch 31/50
35/35 [==============================] - 0s 4ms/step - loss: 92643.9375 - acc: 0.5304
Epoch 32/50
35/35 [==============================] - 0s 4ms/step - loss: 92271.9375 - acc: 0.5286
Epoch 33/50
35/35 [==============================] - 0s 3ms/step - loss: 91910.5547 - acc: 0.5295
Epoch 34/50
35/35 [==============================] - 0s 4ms/step - loss: 91544.7812 - acc: 0.5286
Epoch 35/50
35/35 [==============================] - 0s 3ms/step - loss: 91188.8359 - acc: 0.5313
Epoch 36/50
35/35 [==============================] - 0s 4ms/step - loss: 90827.2344 - acc: 0.5341
Epoch 37/50
35/35 [==============================] - 0s 4ms/step - loss: 90470.1953 - acc: 0.5295
Epoch 38/50
35/35 [==============================] - 0s 4ms/step - loss: 90109.1406 - acc: 0.5386
Epoch 39/50
35/35 [=

...

33/33 [==============================] - 0s 3ms/step - loss: 1282.2321 - acc: 0.7256
Epoch 14/50
33/33 [==============================] - 0s 3ms/step - loss: 1245.6663 - acc: 0.7382
Epoch 15/50
33/33 [==============================] - 0s 3ms/step - loss: 1218.2609 - acc: 0.7411
Epoch 16/50
33/33 [==============================] - 0s 3ms/step - loss: 1185.1702 - acc: 0.7440
Epoch 17/50
33/33 [==============================] - 0s 3ms/step - loss: 1155.6171 - acc: 0.7546
Epoch 18/50
33/33 [==============================] - 0s 3ms/step - loss: 1125.6829 - acc: 0.7507
Epoch 19/50
33/33 [==============================] - 0s 3ms/step - loss: 1100.6572 - acc: 0.7594
Epoch 20/50
33/33 [==============================] - 0s 3ms/step - loss: 1076.5895 - acc: 0.7671
Epoch 21/50
33/33 [==============================] - 0s 3ms/step - loss: 1052.7689 - acc: 0.7671
Epoch 22/50
33/33 [==============================] - 0s 3ms/step - loss: 1035.5693 - acc: 0.7691
Epoch 23/50
33/33 [=======================

33/33 [==============================] - 0s 3ms/step - loss: 665.5837 - acc: 0.8390
Epoch 39/50
33/33 [==============================] - 0s 3ms/step - loss: 660.1245 - acc: 0.8322
Epoch 40/50
33/33 [==============================] - 0s 3ms/step - loss: 648.7677 - acc: 0.8409
Epoch 41/50
33/33 [==============================] - 0s 3ms/step - loss: 641.2318 - acc: 0.8400
Epoch 42/50
33/33 [==============================] - 0s 5ms/step - loss: 636.0687 - acc: 0.8361
Epoch 43/50
33/33 [==============================] - 0s 4ms/step - loss: 629.3771 - acc: 0.8429
Epoch 44/50
33/33 [==============================] - 0s 4ms/step - loss: 623.2899 - acc: 0.8477
Epoch 45/50
33/33 [==============================] - 0s 3ms/step - loss: 621.3582 - acc: 0.8322
Epoch 46/50
33/33 [==============================] - 0s 3ms/step - loss: 609.4901 - acc: 0.8487
Epoch 47/50
33/33 [==============================] - 0s 4ms/step - loss: 604.8020 - acc: 0.8506
Epoch 48/50
33/33 [==============================] -

32/32 [==============================] - 0s 3ms/step - loss: 1404.4321 - acc: 0.6729
Epoch 4/50
32/32 [==============================] - 0s 3ms/step - loss: 1252.4355 - acc: 0.7217
Epoch 5/50
32/32 [==============================] - 0s 3ms/step - loss: 1159.5408 - acc: 0.7402
Epoch 6/50
32/32 [==============================] - 0s 3ms/step - loss: 1093.2463 - acc: 0.7764
Epoch 7/50
32/32 [==============================] - 0s 3ms/step - loss: 1046.3984 - acc: 0.7822
Epoch 8/50
32/32 [==============================] - 0s 3ms/step - loss: 1012.3649 - acc: 0.8037
Epoch 9/50
32/32 [==============================] - 0s 3ms/step - loss: 983.2188 - acc: 0.8057
Epoch 10/50
32/32 [==============================] - 0s 3ms/step - loss: 958.2396 - acc: 0.8086
Epoch 11/50
32/32 [==============================] - 0s 4ms/step - loss: 933.8307 - acc: 0.8193
Epoch 12/50
32/32 [==============================] - 0s 4ms/step - loss: 913.9374 - acc: 0.8242
Epoch 13/50
32/32 [==============================] -

40/40 [==============================] - 0s 4ms/step - loss: 2756.3926 - acc: 0.7202
Epoch 29/50
40/40 [==============================] - 0s 3ms/step - loss: 2720.4507 - acc: 0.7218
Epoch 30/50
40/40 [==============================] - 0s 3ms/step - loss: 2688.3066 - acc: 0.7298
Epoch 31/50
40/40 [==============================] - 0s 3ms/step - loss: 2655.4492 - acc: 0.7274
Epoch 32/50
40/40 [==============================] - 0s 3ms/step - loss: 2620.5293 - acc: 0.7306
Epoch 33/50
40/40 [==============================] - 0s 3ms/step - loss: 2591.0251 - acc: 0.7370
Epoch 34/50
40/40 [==============================] - 0s 3ms/step - loss: 2561.0823 - acc: 0.7458
Epoch 35/50
40/40 [==============================] - 0s 4ms/step - loss: 2529.9324 - acc: 0.7514
Epoch 36/50
40/40 [==============================] - 0s 3ms/step - loss: 2500.0239 - acc: 0.7426
Epoch 37/50
40/40 [==============================] - 0s 3ms/step - loss: 2470.4155 - acc: 0.7450
Epoch 38/50
40/40 [=======================

KeyboardInterrupt: 